In [ ]:
# display_results.ipynb
# =========================================
# Display Bank & Reviews Results with Charts
# =========================================

# 1️⃣ SETUP: Imports & DB Connection
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px

DB_USER = "moggy"
DB_PASSWORD = "MoGGy8080"
DB_HOST = "localhost"
DB_PORT = 5432
DB_NAME = "bank_reviews"

ENGINE = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# 2️⃣ LOAD BANKS & REVIEWS TABLES
banks_df = pd.read_sql("SELECT * FROM banks ORDER BY bank_id;", ENGINE)
reviews_df = pd.read_sql("SELECT * FROM reviews ORDER BY review_date DESC;", ENGINE)

print("📋 Banks Table")
display(banks_df)

print("📋 Latest 20 Reviews")
display(reviews_df.head(20))

# 3️⃣ AGGREGATED ANALYSIS

# Average rating per bank
avg_rating_df = pd.read_sql("""
    SELECT b.bank_name, b.app_name,
           COUNT(r.review_id) AS total_reviews,
           ROUND(AVG(r.rating)::numeric, 2) AS avg_rating
    FROM banks b
    LEFT JOIN reviews r ON b.bank_id = r.bank_id
    GROUP BY b.bank_name, b.app_name
    ORDER BY avg_rating DESC;
""", ENGINE)

print("⭐ Average Ratings per Bank")
display(avg_rating_df)

# Bar chart for average ratings
fig_avg = px.bar(
    avg_rating_df,
    x='bank_name',
    y='avg_rating',
    text='avg_rating',
    title='Average Rating per Bank',
    labels={'bank_name': 'Bank', 'avg_rating': 'Average Rating'},
)
fig_avg.update_traces(textposition='outside', marker_color='teal')
fig_avg.show()

# Sentiment distribution per bank
sentiment_df = pd.read_sql("""
    SELECT b.bank_name, r.sentiment_label, COUNT(*) AS count
    FROM banks b
    LEFT JOIN reviews r ON b.bank_id = r.bank_id
    GROUP BY b.bank_name, r.sentiment_label
    ORDER BY b.bank_name, r.sentiment_label;
""", ENGINE)

print("📊 Sentiment Distribution per Bank")
display(sentiment_df)

# Stacked bar chart for sentiment distribution
fig_sent = px.bar(
    sentiment_df,
    x='bank_name',
    y='count',
    color='sentiment_label',
    text='count',
    title='Sentiment Distribution per Bank',
    labels={'bank_name': 'Bank', 'count': 'Number of Reviews', 'sentiment_label': 'Sentiment'},
)
fig_sent.update_traces(textposition='inside')
fig_sent.show()

# 4️⃣ OPTIONAL: Filter Reviews by Bank
bank_to_view = "Some Bank Name"  # Replace with actual bank_name
filtered_reviews = reviews_df[reviews_df["bank_id"].isin(banks_df[banks_df["bank_name"] == bank_to_view]["bank_id"])]

print(f"📝 Reviews for {bank_to_view}")
display(filtered_reviews)
